## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
PATH = "data/cifar10/"
os.makedirs(PATH,exist_ok=True)

### Downloading CIFAR 10

In [ ]:
from fastai import io

In [ ]:
import tarfile

In [ ]:
def untar_file(file_path, save_path):
    if file_path.endswith('.tar.gz') or file_path.endswith('.tgz'):
        obj = tarfile.open(file_path)
        obj.extractall(save_path)
        obj.close()
        os.remove(file_path)

In [ ]:
cifar_url = 'http://files.fast.ai/data/cifar10.tgz' # faster download
# cifar_url = 'http://pjreddie.com/media/files/cifar.tgz'

io.get_data(cifar_url, 'data/cifar10.tgz')
untar_file('data/cifar10.tgz', 'data/')

### Load classes

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [4]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [5]:
bs=128

### Look at data

In [ ]:
data = get_data(32,4)

In [ ]:
x,y=next(iter(data.trn_dl))

In [ ]:
plt.imshow(data.trn_ds.denorm(x)[0]);

In [ ]:
plt.imshow(data.trn_ds.denorm(x)[1]);

## Initial model

In [25]:
from fastai.models.cifar10.resnext import resnext29_8_64

# m = resnext29_8_64()
m = resnet50(False)
bm = BasicModel(m.half().cuda(), name='cifar10_resnet50')

In [26]:
data = get_data(8,bs*4*4)

In [27]:
data.trn_dl.fp16

False

In [28]:
data.half()

In [29]:
it = iter(data.trn_dl)

In [30]:
x,y = next(it)

Self fp16 True


RuntimeError: tried to construct a tensor from a nested int sequence, but found an item of type numpy.float32 at index (0, 0, 0, 0)

In [12]:
x1,y1 = next(it)

Self fp16 True
FP16: False
Type: <class 'torch.cuda.FloatTensor'>
FP16: False
Type: <class 'torch.cuda.LongTensor'>


In [ ]:
type(x)

In [ ]:
learn = ConvLearner(data, bm)
learn.unfreeze()

In [ ]:
lr=5e-2; wd=5e-4

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
%pdb off

In [ ]:
%time learn.fit(lr, 1, cycle_len=1, use_clr=(20,8))

In [ ]:
%time learn.fit(lr, 2, cycle_len=1)

In [ ]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('8x8_8')

## 16x16

In [ ]:
learn.load('8x8_8')

In [ ]:
learn.set_data(get_data(16,bs*2))

In [ ]:
%time learn.fit(1e-3, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr=1e-2

In [ ]:
%time learn.fit(lr, 2, cycle_len=1, wds=wd)

In [ ]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.save('16x16_8')

## 24x24

In [ ]:
learn.load('16x16_8')

In [ ]:
learn.set_data(get_data(24,bs))

In [ ]:
%time learn.fit(1e-2, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
%time learn.fit(lr, 1, cycle_len=1, wds=wd)

In [ ]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.save('24x24_8')

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)metrics.log_loss(y,preds), accuracy(preds,y)

## 32x32

In [ ]:
learn.load('24x24_8')

In [ ]:
learn.set_data(get_data(32,bs))

In [ ]:
%time learn.fit(1e-2, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
%time learn.fit(lr, 3, cycle_len=4, wds=wd)

In [ ]:
log_preds,y = learn.TTA()
metrics.log_loss(y,np.exp(log_preds)), accuracy(log_preds,y)

In [ ]:
learn.save('32x32_8')